In [1]:
# !wget https://huggingface.co/datasets/malaysia-ai/starcoderdata-sample/resolve/main/rust.jsonl

In [2]:
from huggingface_hub import InferenceClient
from tqdm import tqdm
import os
import json
from glob import glob

/home/husein/.local/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.2.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [3]:
ls = []

with open('rust.jsonl') as fopen:
    for l in tqdm(fopen):
        try:
            l = json.loads(l)
            if len(l['content'].split()) > 5e3:
                continue
            ls.append(l['content'])
        except:
            pass
len(ls)

200000it [00:15, 12960.19it/s]


196310

In [4]:
client = InferenceClient(
    '', timeout = 120
)


def format_prompt(message, history):
  prompt = "<s>"
  for user_prompt, bot_response in history:
    prompt += f"[INST] {user_prompt} [/INST]"
    prompt += f" {bot_response}</s> "
  prompt += f"[INST] {message} [/INST]"
  return prompt

def format_user(history):
    prompt = "<s>"
    for user_prompt, bot_response in history:
        prompt += f"[INST] {user_prompt} [/INST]"
        prompt += f" {bot_response}</s> "
    prompt += f"[INST]"
    return prompt

In [5]:
generate_kwargs = dict(
    temperature=1.0,
    max_new_tokens=5120,
    top_p=0.95,
    repetition_penalty=1.0,
    do_sample=True,
)

In [6]:
!mkdir mixtral-rust
# !rm -rf mixtral-rust/*.json

mkdir: cannot create directory ‘mixtral-rust’: File exists


In [7]:
def answer(q, i):
    filename = f'mixtral-rust/{i}.json'
    if os.path.exists(filename):
        return
    
    while True:
    # for _ in range(5):
        try:
            
            prompt = f"""
Please gain inspiration from the following random code snippet to create a high-quality Rust problem. Present your output in two distinct sections: [Problem Description] and [Solution].

Rust snippet for inspiration:
```
{q}
```

Guidelines for each section:

1. [Problem Description]: This should be **completely self-contained**, providing all the contextual information one needs to understand and solve the problem. Assume common programming knowledge, but ensure that any specific context, variables, or code snippets pertinent to this problem are explicitly included.

2. [Solution]: Offer a comprehensive, **correct** solution that accurately addresses the [Problem Description] you provided.
""".strip()
            formatted_prompt = format_prompt(prompt, [])
            stream = client.text_generation(formatted_prompt, **generate_kwargs, stream=False, details=True, return_full_text=False)
            output = stream.generated_text
            with open(filename, 'w') as fopen:
                json.dump(output, fopen)
            break
        except Exception as e:
            if 'tokens + `max_new_tokens`' in str(e):
                with open(filename, 'w') as fopen:
                    json.dump(False, fopen)
                break
            pass

In [8]:
def consumer(queue, name):
    while True:
        if queue.qsize() == 0:
            break
        item = queue.get()
        answer(*item)
    print(f'consumer {name} done')

In [9]:
from threading import Thread
from queue import Queue

queue = Queue()
questions = ls
urls = [(q, no) for no, q in enumerate(questions)]
for u in urls:
    queue.put(u)
    
ori_size = queue.qsize()

In [10]:
answer(*urls[1])

In [11]:
with open('mixtral-rust/1.json') as fopen:
    print(json.loads(fopen.read()))

 **[Problem Description]**

Rust's random number library `rand` provides a uniform sampling distribution for various numeric types through the `SampleRange` trait. Implementing this trait for custom types enables generating random numbers within a given range (low, high) uniformly. However, implementing it for signed integer types can be surprisingly complex due to edge cases.

You are given a skeleton of a custom `SampleRange` implementation for a hypothetical `MyInt` type, a signed integer that supports common arithmetic and bitwise operations. Your task is to complete the implementation in a way that allows `MyInt` to be used with the `Range` struct, similar to the built-in integer types.

```rust
/// A hypothetical signed integer type with common arithmetic and bitwise operations.
pub struct MyInt(i32);

impl MyInt {
    // Implement `wrapping_add`, `wrapping_sub`, `is_negative`, and `wrapping_neg`
}

impl SampleRange for MyInt {
    fn construct_range(low: Self, high: Self) -> Ran

In [ ]:
max_worker = 50
consumers = [Thread(target=consumer, args=(queue,i)) for i in range(max_worker)]
for i in range(len(consumers)):
    consumers[i].start()
    
pbar = tqdm(total=ori_size)
last_size = 0
while True:
    size = queue.qsize()
    if size == 0:
        break
    left = ori_size - size
    minus = left - last_size
    if minus > 0:
        pbar.update(minus)
        last_size += minus

pbar.close()

  8%|███████▌                                                                                        | 15389/196310 [12:44:24<203:56:21,  4.06s/it]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 10%|██████████                                                                                      | 20502/196310 [18:47:46<370:19:58,  7.58s/it]